<a href="https://colab.research.google.com/github/Ninja7328/ML-Practicing/blob/main/Stacked_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D

In [2]:
# Parameters
max_features  = 20000     # Maximum number of words to consider as features
maxlen = 100              # Maximum length of each sequence
embedding_dim = 128       # Dimension of word embeddings
num_lstm_layers = 2       #Number of LSTM layers
num_lstm_units = 64       # Number of LSTM units in each layer

# Load adn preprocess the IMDB dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

17464789/17464789 [==============================] - 0s 0us/step


In [3]:
# Define the model
model = Sequential()

# Add embedding layer
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(SpatialDropout1D(0.2))

# Add Stacked LSTM layers
for i in range(num_lstm_layers):
  if i == 0:
    # First layer needs input shape
    model.add(LSTM(num_lstm_units, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
  elif i == num_lstm_layers - 1:
    # Last layers doesn't return sequences
    model.add(LSTM(num_lstm_units, dropout=0.2, recurrent_dropout=0.2))
  else:
    # Intermediate layers return sequences
    model.add(LSTM(num_lstm_units, droptout=0.2, recurrent_dropout=0.2, return_sequences=True))


# Add Output layer
model.add(Dense(1, activation='sigmoid'))

In [4]:
#Compile the model
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [5]:
# Print model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          2560000   
                                                                 
 spatial_dropout1d (Spatial  (None, 100, 128)          0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 100, 64)           49408     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2642497 (10.08 MB)
Trainable params: 2642497 (10.08 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [6]:
history = model.fit(x_train, y_train, batch_size=128, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
196/196 [==============================] - 152s 737ms/step - loss: 0.4327 - accuracy: 0.7898 - val_loss: 0.3456 - val_accuracy: 0.8465
Epoch 2/5
196/196 [==============================] - 145s 741ms/step - loss: 0.2494 - accuracy: 0.9019 - val_loss: 0.3468 - val_accuracy: 0.8492
Epoch 3/5
196/196 [==============================] - 142s 725ms/step - loss: 0.1695 - accuracy: 0.9362 - val_loss: 0.3946 - val_accuracy: 0.8399
Epoch 4/5
196/196 [==============================] - 142s 723ms/step - loss: 0.1131 - accuracy: 0.9593 - val_loss: 0.5637 - val_accuracy: 0.8282
Epoch 5/5
196/196 [==============================] - 143s 728ms/step - loss: 0.0741 - accuracy: 0.9739 - val_loss: 0.6629 - val_accuracy: 0.8238
